In [1]:
import cv2 as cv
import os
import numpy as np
import pickle

from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [2]:

class FACELOADING:
    def __init__(self, model_file, encoder_file):
        self.target_size = (100, 100)
        self.detector = MTCNN()
        self.embedder = FaceNet()
        self.model = pickle.load(open(model_file, 'rb'))
        self.encoder = pickle.load(open(encoder_file, 'rb'))

    def recognize_face(self):
        cap = cv.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            if ret:
                frame = cv.flip(frame, 1)  # flip the frame horizontally for a mirror effect
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = self.detector.detect_faces(frame)
                if results:
                    for result in results:
                        x, y, w, h = result['box']
                        x, y = abs(x), abs(y)
                        face = frame[y:y+h, x:x+w]
                        face_arr = cv.resize(face, self.target_size)
                        face_embedded = self.embedder.embeddings(np.asarray([face_arr]))
                        pred_encoded = self.model.predict(face_embedded)
                        pred_proba = self.model.predict_proba(face_embedded)
                        pred_proba = np.max(pred_proba) * 100
                        if pred_encoded in self.encoder.transform(self.encoder.classes_):
                            pred_name = self.encoder.inverse_transform(pred_encoded)
                            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                            cv.putText(frame, f"{pred_name[0]} ({pred_proba:.2f}%)", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                        else:
                            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                            cv.putText(frame, "Unrecognized face: Entry denied", (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                cv.imshow('frame', frame)
                if cv.waitKey(1) == ord('q'):
                    break
            else:
                break
        cap.release()
        cv.destroyAllWindows()

# Define the paths to the pre-trained model and label encoder files
model_file = "trained_model.pkl"
encoder_file = "label_encoder.pkl"

faceloading = FACELOADING(model_file, encoder_file)
faceloading.recognize_face()

c:\Users\91866\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\91866\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 [==============================] - 0s 46ms/step
